In [ ]:
!pip install deepface
#import os
#import shutil
import gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Setup Repository
import os
os.chdir('/content')
CODE_DIR = 'encoder4editing'

!git clone https://github.com/omertov/encoder4editing.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.

%load_ext autoreload
%autoreload 2

In [ ]:
#@title Download
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), CODE_DIR, "pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url   

MODEL_PATHS = {
     "unmasked_masked": {"id": "1eNyb7FyrV0yTjYBL0FKdwN2fqG_2-zRt", "name": "e4e_unmasked_masked.pt"},
}    

path = MODEL_PATHS["unmasked_masked"]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 

!wget {download_command}

In [ ]:
# Setup required image transformations
EXPERIMENT_DATA_ARGS = {
    "unmasked_masked": {
        "model_path": "pretrained_models/e4e_unmasked_masked.pt",
        "image_path": "notebooks/images/1.png"
    }}
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS["unmasked_masked"]
EXPERIMENT_ARGS['transform'] = transforms.Compose([
                              transforms.Resize((256, 256)),
                              transforms.ToTensor(),
                              transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
resize_dims = (256, 256)
img_transforms = EXPERIMENT_ARGS['transform']

In [ ]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
# pprint.pprint(opts)  # Display full options used
# update the training options
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

In [ ]:
def display_alongside_source_image(result_image, source_image):
    res = np.concatenate([np.array(source_image.resize(resize_dims)),
                          np.array(result_image.resize(resize_dims))], axis=1)
    return Image.fromarray(res)

def run_on_batch(inputs, net):
    images, latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
    #if experiment_type == 'cars_encode':
        #images = images[:, :, 32:224, :]
    return images, latents

In [ ]:
#replace file to mute warning
#import gdown
import shutil
url = "https://drive.google.com/u/0/uc?id=1aS5SJCK9MLMyYPXmhAkwtn3cJhQGCbGj&export=download"
gdown.download(url, "/content/sample_data/DeepFace.py")
os.remove("/usr/local/lib/python3.7/dist-packages/deepface/DeepFace.py")
shutil.move("/content/sample_data/DeepFace.py","/usr/local/lib/python3.7/dist-packages/deepface/")

In [ ]:
!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_test_unmasked_folder.zip -d /content/sample_data
!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_test_masked_folder.zip -d /content/sample_data

In [ ]:
#df['VGG-Face_cosine'][0]
#00131.png
#masked  -----  0.3056197013748233
#predicted ----  0.261993
#ground_truth -  9.992007e-16
!mkdir /content/sample_data/result_face

In [ ]:
def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance  

In [ ]:
!pip uninstall deepface
!pip install deepface
from deepface import DeepFace
import gdown

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
from torchvision.utils import save_image
#from deepface import DeepFace
masked_dir = "/content/sample_data/AAFD_test_masked_folder/"
db_path = "/content/sample_data/AAFD_test_unmasked_folder"
count = 0
total_accuracy = 0
for masked_image_dir in os.listdir(masked_dir):
  if not masked_image_dir.endswith(".ipynb_checkpoints"):
    count += 1
    image = Image.open(masked_dir + masked_image_dir)
    image = image.convert("RGB")
    image = image.resize(resize_dims)
    image = img_transforms(image)
    with torch.no_grad():
      tic = time.time()
      images, latents = run_on_batch(image.unsqueeze(0), net)
      result_image, latent = images[0], latents[0]
      result_image = (result_image+1)/2
      result_dir = "/content/sample_data/result_face/"+ masked_image_dir
      save_image(result_image,result_dir)
      toc = time.time()
      df = DeepFace.find(img_path=result_dir,db_path=db_path,enforce_detection=False)
      accuracy = 1 - calc_distance(masked_image_dir.split(".")[0],df)
      total_accuracy += accuracy
      print("masked_image :",masked_image_dir,"recognize_image:",df['identity'][0].split("/")[4],"Accuracy: ",accuracy)
    if(count >= 1000):
      break  

print("Accuracy of Model over {} images is : {}".format(count,total_accuracy/count))

In [ ]:
#celeba e4e 0.13463303921646666

In [ ]:
df

In [ ]:
from torchvision.utils import save_image
def unmasked_image(path):
  #read image
  image = Image.open("/content/sample_data/ffhq_validation_set/masked/"+path)
  image = image.convert("RGB")
  #transform first
  image = image.resize(resize_dims)
  image = img_transforms(image)
  #unmasked now
  with torch.no_grad():
    tic = time.time()
    images, latents = run_on_batch(image.unsqueeze(0), net)
    result_image, latent = images[0], latents[0]
    result_image = (result_image+1)/2
    save_image(result_image,"/content/sample_data/result_face/"+path)
    toc = time.time()
    print('Inference took {:.4f} seconds.'.format(toc - tic))

unmasked_image("00132.png")    

In [ ]:
unmasked_image()